<a href="https://colab.research.google.com/github/Lakshitalearning/HireSync-AI/blob/main/ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertModel.from_pretrained("bert-base-multilingual-cased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
def get_mbert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state[:, 0, :].squeeze()
    return embeddings

In [ ]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
job_description = " ".join(employers[0]["required_skills"] + employers[0]["other_requirements"])
candidate_profile = " ".join(candidates[0]["skills"] + [candidates[0]["education"]] + [str(candidates[0]["experience"])])
vectorizer.fit([job_description, candidate_profile])

with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

with open("scoring_weights.pkl", "wb") as f:
    pickle.dump(weights, f)

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertModel.from_pretrained("bert-base-multilingual-cased")

def get_mbert_embeddings(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Generate embeddings using mBERT
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the [CLS] token embedding as the sentence embedding
    embeddings = outputs.last_hidden_state[:, 0, :].squeeze()
    return embeddings

# Structured data for employers and candidates (same as before)
employers = [
    {
        "title": "Senior WordPress Developer",
        "required_skills": ["WordPress", "PHP", "HTML5", "CSS3", "JavaScript", "MySQL", "Git", "server setup", "DNS", "React", "Vue.js", "Angular"],
        "required_education": "B.Tech",
        "location": "Greater Noida",
        "experience": 1,
        "salary_range": (500000, 600000),
        "other_requirements": ["SEO", "accessibility standards", "AWS", "Google Cloud", "Azure"]
    },
    {
        "title": "Graphic Designer",
        "required_skills": ["Figma", "Adobe Illustrator", "Adobe Photoshop", "InDesign", "HTML", "CSS"],
        "required_education": None,
        "location": "Noida",
        "experience": 1,
        "salary_range": (200000, 300000),
        "other_requirements": ["web design principles"]
    },
    {
        "title": "Cinematographer",
        "required_skills": ["Adobe Premiere Pro", "Adobe Photoshop", "video editing", "photography"],
        "required_education": None,
        "location": None,
        "experience": 0,
        "salary_range": (200000, 300000),
        "other_requirements": ["creative storytelling"]
    },
    {
        "title": "Financial Sales Officer",
        "required_skills": ["English Proficiency (Spoken)", "customer service", "sales"],
        "required_education": "Any graduate",
        "location": ["Kurnool", "Visakhapatnam", "Hyderabad"],
        "experience": 0,
        "salary_range": (400000, 430000),
        "other_requirements": ["banking finance"]
    },
    {
        "title": "Associate Content Writer",
        "required_skills": ["SEO", "Creative Writing", "Blogging", "Digital Marketing", "Social Media Marketing"],
        "required_education": None,
        "location": None,
        "experience": 0,
        "salary_range": (200000, 400000),
        "other_requirements": ["content management systems"]
    }
]

# Structured data for all 16 candidates
candidates = [
    {
        "name": "PERSON1",
        "skills": ["Python", "PHP", "JavaScript", "React", "MySQL", "Git", "Docker", "Apache"],
        "education": "B.Tech",
        "experience": 2,
        "desired_salary": 1000000,
        "location": "Mangalore, Karnataka",
        "willing_to_relocate": False
    },
    {
        "name": "PERSON2",
        "skills": ["WordPress", "PHP", "HTML", "CSS", "JavaScript"],
        "education": "12th Pass",
        "experience": 15,
        "desired_salary": 400000,
        "location": "San Francisco, CA",
        "willing_to_relocate": False
    },
    {
        "name": "PERSON3",
        "skills": ["WordPress", "PHP", "JavaScript", "HTML", "CSS", "MySQL"],
        "education": "Bachelor's Degree in Computer Science",
        "experience": 10,
        "desired_salary": None,
        "location": "Detroit, MI",
        "willing_to_relocate": False
    },
    {
        "name": "PERSON4",
        "skills": ["Adobe Creative Suite", "Figma", "Sketch", "HTML5", "CSS3"],
        "education": "Master of Fine Arts - Graphic Design",
        "experience": 8,
        "desired_salary": None,
        "location": "San Francisco, CA",
        "willing_to_relocate": False
    },
    {
        "name": "PERSON5",
        "skills": ["Adobe Creative Suite", "Figma", "Sketch", "HTML5", "CSS3", "JavaScript"],
        "education": "Master of Fine Arts - Graphic Design",
        "experience": 5,
        "desired_salary": None,
        "location": "New York, NY",
        "willing_to_relocate": True
    },
    {
        "name": "PERSON6",
        "skills": ["Adobe Creative Suite", "Sketch", "InVision", "Figma", "HTML5", "CSS3"],
        "education": "Master of Fine Arts - Graphic Design",
        "experience": 7,
        "desired_salary": None,
        "location": "Los Angeles, CA",
        "willing_to_relocate": True
    },
    {
        "name": "PERSON7",
        "skills": ["Sales", "Customer Service", "Investment Strategies", "Financial Analysis"],
        "education": "MBA - Finance",
        "experience": 10,
        "desired_salary": None,
        "location": "Seattle, WA",
        "willing_to_relocate": False
    },
    {
        "name": "PERSON8",
        "skills": ["Financial Analysis", "Excel", "SQL", "Tableau", "Budgeting"],
        "education": "MBA - Finance",
        "experience": 5,
        "desired_salary": None,
        "location": "New York, NY",
        "willing_to_relocate": True
    },
    {
        "name": "PERSON9",
        "skills": ["Financial Modeling", "Python", "Excel", "SQL", "Risk Management"],
        "education": "MBA - Finance",
        "experience": 8,
        "desired_salary": None,
        "location": "Chicago, IL",
        "willing_to_relocate": True
    },
    {
        "name": "PERSON10",
        "skills": ["Financial Analysis", "Budgeting", "Risk Management", "SQL", "Tableau"],
        "education": "MBA - Finance",
        "experience": 12,
        "desired_salary": None,
        "location": "San Francisco, CA",
        "willing_to_relocate": False
    },
    {
        "name": "PERSON11",
        "skills": ["SEO", "Creative Writing", "Blogging", "Content Marketing"],
        "education": "MBA - Marketing",
        "experience": 4,
        "desired_salary": None,
        "location": None,
        "willing_to_relocate": True
    },
    {
        "name": "PERSON12",
        "skills": ["Content Writing", "SEO", "Social Media Marketing", "Blogging"],
        "education": "Master of Arts - New Media",
        "experience": 6,
        "desired_salary": None,
        "location": "Los Angeles, CA",
        "willing_to_relocate": True
    },
    {
        "name": "PERSON13",
        "skills": ["Content Writing", "SEO", "Social Media Marketing", "Blogging"],
        "education": "Master of Arts - English",
        "experience": 5,
        "desired_salary": None,
        "location": "Chicago, IL",
        "willing_to_relocate": True
    },
    {
        "name": "PERSON14",
        "skills": ["Photography", "Adobe Photoshop", "Lightroom", "Photo Editing"],
        "education": "Master of Fine Arts - Photography",
        "experience": 7,
        "desired_salary": None,
        "location": "New York, NY",
        "willing_to_relocate": True
    },
    {
        "name": "PERSON15",
        "skills": ["Photography", "Adobe Photoshop", "Lightroom", "Photo Editing"],
        "education": "Bachelor of Fine Arts - Photography",
        "experience": 5,
        "desired_salary": None,
        "location": "Los Angeles, CA",
        "willing_to_relocate": True
    },
    {
        "name": "PERSON16",
        "skills": ["Photography", "Adobe Photoshop", "Lightroom", "Photo Editing"],
        "education": "Bachelor of Fine Arts - Photography",
        "experience": 6,
        "desired_salary": None,
        "location": "San Francisco, CA",
        "willing_to_relocate": False
    }
]


# Scoring weights
weights = {
    "skills": 0.4,
    "experience": 0.2,
    "education": 0.15,
    "location": 0.15,
    "salary": 0.1
}

def calculate_match_score(job, candidate):
    score = 0

    # Skills match using mBERT embeddings and cosine similarity
    job_description = " ".join(job["required_skills"] + job["other_requirements"])
    candidate_profile = " ".join(candidate["skills"] + [candidate["education"]] + [str(candidate["experience"])])

    # Get mBERT embeddings for job description and candidate profile
    job_embedding = get_mbert_embeddings(job_description)
    candidate_embedding = get_mbert_embeddings(candidate_profile)

    # Calculate cosine similarity between embeddings
    cosine_sim = cosine_similarity(job_embedding.unsqueeze(0), candidate_embedding.unsqueeze(0))
    skills_score = cosine_sim[0][0]
    score += skills_score * weights["skills"]

    # Experience match
    exp_score = 1 if candidate["experience"] >= job["experience"] else 0
    score += exp_score * weights["experience"]

    # Education match
    edu_score = 0
    if job["required_education"]:
        if candidate["education"] and job["required_education"].lower() in candidate["education"].lower():
            edu_score = 1
    else:
        edu_score = 1  # No education requirement
    score += edu_score * weights["education"]

    location_score = 0
    if job["location"]:
        if candidate["willing_to_relocate"]:
            location_score = 1
        elif candidate["location"]:
            if any(loc.lower() in candidate["location"].lower() for loc in job["location"] if isinstance(job["location"], list)) or \
               (isinstance(job["location"], str) and job["location"].lower() in candidate["location"].lower()):
                location_score = 1
        else:
            location_score = 0.5  # Location not specified
    else:
        location_score = 1  # No location requirement
    score += location_score * weights["location"]

    # Salary match
    salary_score = 0
    if candidate["desired_salary"] and job["salary_range"]:
        if (candidate["desired_salary"] >= job["salary_range"][0]) and (candidate["desired_salary"] <= job["salary_range"][1]):
            salary_score = 1
        elif candidate["desired_salary"] < job["salary_range"][0]:
            salary_score = 0.8  # Underpaid (acceptable)
        else:
            salary_score = 0.2  # Overpaid (penalty)
    else:
        salary_score = 0.5  # Missing data
    score += salary_score * weights["salary"]

    return round(score * 100, 2)

for job in employers:
    print(f"Job: {job['title']}")
    ranked = []
    for candidate in candidates:
        score = calculate_match_score(job, candidate)
        ranked.append((candidate["name"], score))

    ranked.sort(key=lambda x: x[1], reverse=True)
    for name, score in ranked:
        print(f"- {name}: Score {score}%")
    print("\n")

with open("scoring_weights.pkl", "wb") as f:
    pickle.dump(weights, f)

Job: Senior WordPress Developer
- PERSON6: Score 74.56%
- PERSON5: Score 74.28%
- PERSON1: Score 72.31%
- PERSON9: Score 70.44%
- PERSON11: Score 70.12%
- PERSON8: Score 68.99%
- PERSON14: Score 68.93%
- PERSON12: Score 68.7%
- PERSON15: Score 67.78%
- PERSON13: Score 64.48%
- PERSON2: Score 60.37%
- PERSON4: Score 59.55%
- PERSON10: Score 55.83%
- PERSON16: Score 53.18%
- PERSON3: Score 53.02%
- PERSON7: Score 48.94%


Job: Graphic Designer
- PERSON8: Score 69.94%
- PERSON6: Score 69.83%
- PERSON9: Score 69.5%
- PERSON5: Score 69.33%
- PERSON15: Score 69.26%
- PERSON13: Score 69.02%
- PERSON12: Score 68.66%
- PERSON14: Score 68.54%
- PERSON11: Score 67.18%
- PERSON7: Score 58.11%
- PERSON3: Score 56.27%
- PERSON4: Score 54.11%
- PERSON10: Score 54.08%
- PERSON16: Score 53.88%
- PERSON1: Score 53.16%
- PERSON2: Score 49.48%


Job: Cinematographer
- PERSON6: Score 82.55%
- PERSON4: Score 81.97%
- PERSON5: Score 81.33%
- PERSON14: Score 80.6%
- PERSON11: Score 79.53%
- PERSON12: Score 79